<h1 id="tocheading">Table of Contents and Notebook Setup</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd

# Introduction

pandas has a number of functions to read tabular data and convert it into a DataFrame. The most popular ones are <b> read_csv</b> and <b> read_table. </b> csv uses comma as a delimiter and table uses tab as a delimiter.

These functions essentially convert text into a DataFrame. The optional arguments for these functions fall into the following categories.

<u> Indexing </u> - Can treat one or more columns as the returned DataFrame and how to get the column names.

<u> Type inference and data conversion </u> - User defined value conversions and custom list of missing value markers.

<u> Datetime parsing </u> - Allows one to combine date and time information spread over multiple columns into a single column.

<u> Iterating </u> - Support for iterating over large files.

<u> Unclean data issues </u> - Skipping rows or a footer, comments, or other minor things like numeric data with thousands seperated by column (French People)

## Comma Seperated (CSV) Text Files

In [3]:
df = pd.read_csv('sampledata/data1.csv')
df

,Month,People,Sales
0,January,1200,1678
1,March,600,567
2,June,2300,2400
3,August,2333,4000


We could have also used the <b> read_table </b> function and specified the delimiter.

In [4]:
pd.read_table('sampledata/data1.csv', sep=',')

,Month,People,Sales
0,January,1200,1678
1,March,600,567
2,June,2300,2400
3,August,2333,4000


We can also specify the column names ourselves.

In [5]:
pd.read_csv('sampledata/data1.csv', names=['month', 'people', 'sales'])

,month,people,sales
0,Month,People,Sales
1,January,1200,1678
2,March,600,567
3,June,2300,2400
4,August,2333,4000


Note that this data frame already has column header specified, so using this function is required.

## Choose a Specific Column to use as Row Indices

We can select one of the column to use as a row index as follows:

In [6]:
pd.read_csv('sampledata/data1.csv', index_col='Month')

,People,Sales
Month,,
January,1200,1678
March,600,567
June,2300,2400
August,2333,4000


## Text Files of Bizzare Formats

Sometimes data is stored in text files of weird format. They may have variable whitespace and no comma seperated values. Consider the following file.

In [7]:
list(open('sampledata/data2.txt'))

['      People    Sales\n',
 'Jan   12         1200\n',
 'Feb   23         2100\n',
 'Mar   45         4500']

While we could scrape together some code to try and open this and store it in a DataFrame, the <b> read_table </b> function has ways of opening files like this in a simple manner:

In [8]:
pd.read_table('sampledata/data2.txt', sep='\s+')

,People,Sales
Jan,12,1200
Feb,23,2100
Mar,45,4500


In this case, we have passed a regular expression \s+ as a delimiter for the variable whitespace in this text file.

## Skipping Rows and Columns

We can also skip rows and columns of text file data.

In [9]:
pd.read_csv('sampledata/data1.csv', index_col='Month', skiprows=[1,3])

,People,Sales
Month,,
March,600,567
August,2333,4000


In [10]:
pd.read_csv('sampledata/data1.csv', index_col='Month', usecols=['Month','People'])

,People
Month,
January,1200
March,600
June,2300
August,2333


## Handling Missing Values

Suppose our comma delimted data looks something like this (ignore the strange symbols at the beginning). Whenever pandas sees an 'NA' or a blank cell, the corresponding cell in the DataFrame will be NaN.

In [11]:
list(open('sampledata/data3.csv'))

['ï»¿something,a,b,c,message\n',
 'one,12,23,NA,hello\n',
 'two,23,,12,world\n',
 'three,3,,5,foo\n']

Opened as a DataFrame:

In [12]:
pd.read_csv('sampledata/data3.csv')

,something,a,b,c,message
0,one,12,23.0,NaN,hello
1,two,23,NaN,12.0,world
2,three,3,NaN,5.0,foo


Different NA sentinels can be specified for each column when searching for NA values.

In [13]:
sentinels = {'message': ['foo', 'world'], 'something': ['two']}
pd.read_csv('sampledata/data3.csv', na_values = sentinels)

,something,a,b,c,message
0,one,12,23.0,NaN,hello
1,NaN,23,NaN,12.0,NaN
2,three,3,NaN,5.0,NaN


# Reading Text Files in Pieces

Sometimes when processing very large files OR when you want to figure out the proper arguments to correctly process a large file, we may only want to read in a smaller part of the file.

In [14]:
pd.options.display.max_rows = 10

In [15]:
pd.read_csv('sampledata/data4.csv')

,one,two,three,four
0,123,123,546,1
1,12,45,456,234
2,1435,234,456,34
3,3451,65,235,21
4,123,423,23,34
...,...,...,...,...
220,123,123,546,1
221,12,45,456,234
222,1435,234,456,34
223,3451,65,235,21


If we only want to read a small number of rows, we can specify that with the <b> nrows </b> argument.

In [16]:
pd.read_csv('sampledata/data4.csv', nrows = 5)

,one,two,three,four
0,123,123,546,1
1,12,45,456,234
2,1435,234,456,34
3,3451,65,235,21
4,123,423,23,34


We can also use TextParser objects to iterate through parts of the file according to a certain <b> chunksize. </b>

In [17]:
chunker = pd.read_csv('sampledata/data4.csv', nrows=10, chunksize = 5)
chunker

Rather than a DataFrame. the new type of object returned is a TextParser Object. Notice that the file opened above containing 10 rows is being iterated through in chunks of 5 rows at a time.

In [18]:
for piece in chunker:
    print(piece)
    print('---------------------------')

    one  two  three  four
0   123  123    546     1
1    12   45    456   234
2  1435  234    456    34
3  3451   65    235    21
4   123  423     23    34
---------------------------
    one  two  three  four
5   123  123    546     1
6    12   45    456   234
7  1435  234    456    34
8  3451   65    235    21
9   123  423     23    34
---------------------------
